In [1]:
import numpy as np
import pandas as pd

%load_ext rpy2.ipython

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import os

def load_smooth_dev():
    """Load the smooth package in development mode"""
    smooth_path = "/home/filtheo/smooth/"
    
    ro.r(f'''
    if (!requireNamespace("devtools", quietly=TRUE)) {{
        install.packages("devtools", repos="https://cran.rstudio.com/")
    }}
    devtools::load_all("{smooth_path}")
    ''')
    
    print("Smooth package loaded in development mode")

load_smooth_dev()


ℹ Loading smooth
Exports from /home/filtheo/smooth/src/adamGeneral.cpp:

Exports from /home/filtheo/smooth/src/matrixPowerWrap.cpp:
   RcppExport SEXP matrixPowerWrap(SEXP matA, SEXP power)

Exports from /home/filtheo/smooth/src/ssGeneral.cpp:
   RcppExport SEXP forecasterwrap(SEXP matvt, SEXP matF, SEXP matw, SEXP h, SEXP Etype, SEXP Ttype, SEXP Stype, SEXP lagsModel, SEXP matxt, SEXP matat, SEXP matFX)

Exports from /home/filtheo/smooth/src/ssOccurrence.cpp:
   RcppExport SEXP occurenceFitterWrap(SEXP matvt, SEXP matF, SEXP matw, SEXP vecg, SEXP ot, SEXP modellags, SEXP Etype, SEXP Ttype, SEXP Stype, SEXP Otype, SEXP matxt, SEXP matat, SEXP matFX, SEXP vecgX)
   RcppExport SEXP occurrenceOptimizerWrap(SEXP matvt, SEXP matF, SEXP matw, SEXP vecg, SEXP ot, SEXP modellags, SEXP Etype, SEXP Ttype, SEXP Stype, SEXP Otype, SEXP matxt, SEXP matat, SEXP matFX, SEXP vecgX, SEXP bounds)
   RcppExport SEXP occurenceGeneralFitterWrap(SEXP ot, SEXP modellagsA, SEXP EtypeA, SEXP TtypeA, SEXP Stype

R[write to console]: Loading required package: greybox

R[write to console]: Package "greybox", v2.0.6 loaded.


R[write to console]: This is package "smooth", v4.4.0


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/lib/R/site-library’ contains no packages



Smooth package loaded in development mode


# Two-Stage Initialization Tests - R Implementation

Two-stage initialization works by:
1. First running a model with `initial="two-stage"` (full backcasting) to get good starting values
2. Using those values as initial guesses for optimization, allowing parameter refinement

This should produce results that are:
- Different from pure backcasting (since parameters are refined)
- Different from pure optimal (since starting values are better)
- Generally better or similar quality to both methods


### Test 1: Global level ETS(A,N,N) - two-stage vs optimal vs backcasting


In [2]:
np.random.seed(33)
n_points = 100
time_series = np.random.normal(100, 10, n_points)
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [3]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'optimal')
cat('ETS(A,N,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)


ETS(A,N,N) with optimal initial:
Parameters: 0.02502502 98.50918 
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 101.134 101.134 101.134 101.134 101.134 101.134 101.134 101.134 101.134
[10] 101.134 101.134 101.134


In [4]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'backcasting', nIterations = 2)
cat('ETS(A,N,N) with backcasting (nIterations=2):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,N,N) with backcasting (nIterations=2):
Parameters: 0 
$mean
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911

$lower
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [5]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'complete')
cat('ETS(A,N,N) with complete (nIterations=2):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,N,N) with complete (nIterations=2):
Parameters: 0 
$mean
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911

$lower
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 101 
End = 112 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [6]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,N,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)


ETS(A,N,N) with two-stage initialization (two-stage):
Parameters: 0 99.61911 
Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911 99.61911
 [9] 99.61911 99.61911 99.61911 99.61911


### Test 2: Local trend ETS(A,A,N)


In [7]:
np.random.seed(42)
n_points = 120
errors = np.random.normal(0, 10, n_points)
trend = np.random.normal(0.5, 2, n_points)
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series})


In [8]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), initial = 'optimal')
forecast(model, h = 12)[1:3] 


$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 164.8028 165.3411 165.8794 166.4177 166.9560 167.4943 168.0326 168.5709
 [9] 169.1092 169.6475 170.1858 170.7241

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [9]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), initial = 'complete')
cat('ETS(A,A,N) with complete initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] 


ETS(A,A,N) with complete initial:
Alpha, Beta, Gamma: 0.198619 0 NA 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.0450 165.6303 166.2156 166.8009 167.3862 167.9715 168.5568 169.1421
 [9] 169.7274 170.3127 170.8980 171.4833

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [10]:
%%R -i ts_df

model <- adam(ts_df, model = "AAN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,A,N) with two-stage initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,N) with two-stage initial:
Alpha, Beta, Gamma: 0.4522367 0 69.08323 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 165.4290 165.4284 165.4277 165.4271 165.4265 165.4258 165.4252 165.4246
 [9] 165.4240 165.4233 165.4227 165.4221

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 3: Seasonal data ETS(A,A,A)


In [11]:
np.random.seed(42)
n_points = 120
errors = (1+np.random.normal(0, 0.1, n_points))
trend = np.random.normal(0.5, 2, n_points)
seasonal_sd = 0.2
seasonal_pattern = np.exp(np.random.normal(0, seasonal_sd, 12))
seasonal_pattern = seasonal_pattern / np.mean(seasonal_pattern)
time_series = np.zeros(n_points)
time_series[0] = 200 * seasonal_pattern[0] * errors[0]
for i in range(n_points-1):
    time_series[i+1] = ((time_series[i] / seasonal_pattern[(i) % 12]-trend[i]) * errors[i] ** (0.1-1) + trend[i+1]) * seasonal_pattern[(i+1) % 12] * errors[i+1]
ts_df = pd.DataFrame({'value': time_series})


In [12]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(A,A,A) with optimal initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with optimal initial:
Alpha, Beta, Gamma: 0.07879636 0.01339435 0.0006640582 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 143.1427 156.6723 171.7813 206.7483 128.0838 167.0542 155.8859 151.7468
 [9] 242.7502 182.7711 137.4582 210.6907

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [13]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(A,A,A) with backcasting initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with backcasting initial:
Alpha, Beta, Gamma: 0.1015004 0 0.07076862 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 143.3342 156.8877 172.4576 205.3987 125.2086 162.9521 154.3471 147.7931
 [9] 239.0042 179.8656 133.4818 202.6318

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [14]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'complete')
cat('ETS(A,A,A) with complete initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with complete initial:
Alpha, Beta, Gamma: 0.1015004 0 0.07076862 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 143.3342 156.8877 172.4576 205.3987 125.2086 162.9521 154.3471 147.7931
 [9] 239.0042 179.8656 133.4818 202.6318

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [15]:
%%R -i ts_df

model <- adam(ts_df, model = "AAA", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(A,A,A) with two-stage initialization (two-stage):\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,A,A) with two-stage initialization (two-stage):
Alpha, Beta, Gamma: 0.1892951 0 0.0004069391 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 149.2699 163.0270 177.7961 210.7980 132.6765 170.2933 162.6390 155.4843
 [9] 245.6806 186.2477 141.6571 213.2781

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 4: Damped trend ETS(A,Ad,N)


In [16]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'optimal')
cat('ETS(A,Ad,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with optimal initial:
Parameters: 0.009354116 0.009347926 0.9234458 207.1153 -0.5876486 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 168.3776 168.6882 168.9751 169.2400 169.4846 169.7105 169.9191 170.1118
 [9] 170.2897 170.4539 170.6056 170.7457

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [17]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'backcasting')
cat('ETS(A,Ad,N) with backcasting initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with backcasting initial:
Parameters: 0.002900778 0 1 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 167.5548 167.3476 167.1404 166.9332 166.7260 166.5189 166.3117 166.1045
 [9] 165.8973 165.6901 165.4829 165.2757

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [18]:
%%R -i ts_df

model <- adam(ts_df, model = "AAdN", lags = c(12), initial = 'two-stage', nIterations = 2)
cat('ETS(A,Ad,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(A,Ad,N) with two-stage initialization (two-stage):
Parameters: 0 0 0.8901187 176.5499 0.2853816 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 178.8617 178.8617 178.8617 178.8617 178.8617 178.8617 178.8617 178.8617
 [9] 178.8617 178.8617 178.8617 178.8617

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 5: Multiplicative error ETS(M,N,N)


In [19]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,N,N) with optimal initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with optimal initial:
Parameters: 0.03293295 189.757 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432 174.2432
 [9] 174.2432 174.2432 174.2432 174.2432

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [20]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(M,N,N) with backcasting initial:\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with backcasting initial:
Parameters: 0.03312988 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972 174.1972
 [9] 174.1972 174.1972 174.1972 174.1972

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [21]:
%%R -i ts_df

model <- adam(ts_df, model = "MNN", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(M,N,N) with two-stage initialization (two-stage):\n')
cat('Parameters:', model$B, '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,N,N) with two-stage initialization (two-stage):
Parameters: 0.03298887 1058.789 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 174.2383 174.2383 174.2383 174.2383 174.2383 174.2383 174.2383 174.2383
 [9] 174.2383 174.2383 174.2383 174.2383

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



### Test 6: Multiplicative seasonal ETS(M,A,M)


In [22]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'optimal')
cat('ETS(M,A,M) with optimal initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with optimal initial:
Alpha, Beta, Gamma: 0.1093149 0.004294988 0.0008970466 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 144.8282 158.6545 174.3859 201.5817 129.2241 165.5948 155.2905 151.8311
 [9] 236.3300 178.3185 138.8740 206.9557

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [23]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'backcasting')
cat('ETS(M,A,M) with backcasting initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with backcasting initial:
Alpha, Beta, Gamma: 0.09784527 0 0.06255799 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 145.3574 158.1146 172.6357 203.2005 128.1912 163.6339 155.4478 149.3114
 [9] 235.6732 179.5255 135.7339 200.4367

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [24]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'complete')
cat('ETS(M,A,M) with complete initial:\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with complete initial:
Alpha, Beta, Gamma: 0.09784527 0 0.06255799 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 145.3574 158.1146 172.6357 203.2005 128.1912 163.6339 155.4478 149.3114
 [9] 235.6732 179.5255 135.7339 200.4367

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



In [25]:
%%R -i ts_df

model <- adam(ts_df, model = "MAM", lags = c(12), distribution = 'dnorm', initial = 'two-stage', nIterations = 2)
cat('ETS(M,A,M) with two-stage initialization (two-stage):\n')
cat('Alpha, Beta, Gamma:', model$B[1:3], '\n')
forecast(model, h = 12)[1:3] |> print()


ETS(M,A,M) with two-stage initialization (two-stage):
Alpha, Beta, Gamma: 0.2000068 0 6.662157e-05 
$mean
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 150.2142 165.0116 180.8035 209.1157 134.0871 171.0784 161.4119 157.6203
 [9] 247.3512 185.9991 143.3390 214.8281

$lower
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA

$upper
Time Series:
Start = 121 
End = 132 
Frequency = 1 
      Series 1
 [1,]       NA
 [2,]       NA
 [3,]       NA
 [4,]       NA
 [5,]       NA
 [6,]       NA
 [7,]       NA
 [8,]       NA
 [9,]       NA
[10,]       NA
[11,]       NA
[12,]       NA



## Summary

Two-stage initialization successfully:
- Runs a two-stage backcasting model first
- Extracts parameters (persistence, damping, ARMA)
- Extracts and normalizes initial states
- Uses these as starting values for optimization
- Produces results that combine benefits of both approaches
